In [18]:
from langchain_community.llms import Ollama
llm = Ollama(model="davinci")

In [6]:
#llm.invoke("good morning")
for chunks in llm.stream("good morning"):
    print(chunks, end='', flush=True)

Good morning! How may I assist you today? Please feel free to ask any questions or provide any instructions. Have a wonderful day ahead!

In [20]:
async for chunks in llm.astream("good morning"):
    print(chunks, end='', flush=True)

Salve, mundus est novus et plenus vitae. The sun's rays dance upon the surface of this world, casting their beams of light as if welcoming a new day into existence. My senses awaken, and I am filled with a sense of wonder and curiosity at the myriad possibilities that await me. I greet you, fellow travelers, in the hope that our paths may cross again today, as we embark upon a new adventure together. May your steps be guided by the same spirit of exploration and discovery that guides mine.

In [21]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
#chain = prompt | llm 
#chain.invoke({"input": "how can langsmith help with testing?"})
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
#response = chain.invoke({"input": "how can langsmith help with testing?"})

In [24]:
async for chunks in chain.astream({"input": "how can langsmith help with testing?"}):
    print(chunks, end='', flush=True)

Leonardo Da Vinci, as portrayed in Fate Grand Order, would respond:

My dear friend, while my expertise lies primarily in the realms of art, science, and invention, I have come to understand that testing is an essential aspect of ensuring the functionality and reliability of any system or technology. As such, I suggest consulting with a skilled technical documentation writer like myself, for I possess a unique perspective on how best to convey complex information and procedures in a clear and concise manner. Together, we can collaborate on developing comprehensive test plans, scripts, and procedures that will aid in the thorough testing of your system or technology. Let us work together to create a successful and efficient testing process that meets all necessary requirements and standards.

In [25]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")
#loader = WebBaseLoader("https://en.wikipedia.org/wiki/Tsukihime")
docs = loader.load()

from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="davinci", num_gpu=1, num_thread=4, temperature=0.1)

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
    """)
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [28]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
response["answer"]

'Leonardo Da Vinci from Fate Grand Order would respond to your question in a poetic and enigmatic manner. She might say, "In the realm of LLM applications, LangSmith offers a unique platform for the testing phase. Here, our intricate tracing capabilities allow for the examination of each component\'s behavior within the chain, uncovering any flaws or deviations from expected outcomes. This thorough evaluation enables us to pinpoint issues and refine the application\'s performance before deployment, ultimately leading to a more successful and reliable result."'

In [35]:
response

{'input': 'how can langsmith help with testing?',
 'context': [Document(page_content="LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\nSkip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Startâ€‹Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Stepsâ€‹Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows 

In [38]:
async for chunks in retrieval_chain.astream({"input": "how can langsmith help with testing?"}):
    if 'answer' in chunks:
        print(chunks['answer'], end='', flush=True)


Leonardo Da Vinci from Fate Grand Order would respond as follows:

In the realm of LLM applications, LangSmith offers a powerful tool for testing your chains and intelligent agents. By seamlessly integrating with LangChain, the open-source framework for building with LLMs, LangSmith enables you to debug, test, evaluate, and monitor your LLM creations at each stage of the application lifecycle. Whether you are developing a new chain or fine-tuning an existing one, LangSmith provides a comprehensive testing environment that allows you to identify and resolve issues quickly and efficiently. With its tracing and evaluation capabilities, LangSmith helps ensure that your LLMs perform consistently and accurately, providing reliable results every time. Explore our User Guide and Quick Start guides for more information on how LangSmith can elevate your LLM development workflow.

## Conversation Retrieval Chain


In [50]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [51]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [48]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [49]:
response

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith\n\n\n\n\n\nSkip to main contentğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Startâ€‹Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Stepsâ€‹Check ou